In [ ]:
!pip install mrsimulator lmfit

# Least-squares fit

A least squares analysis is a three step process

- create a **fitting model**,
- create the **model parameters**, and
- define a **cost function** to minimize.

Here, we will use the mrsimulator objects to create a fitting model, and use the [LMFIT](https://lmfit.github.io/lmfit-py/) library for performing the least-squares fitting optimization.

In this example, we use a synthetic $^{29}$Si NMR spectrum of cuspidine, generated from the tensor parameters reported by Hansen *et. al.* [1], to demonstrate a simple fitting procedure.

In [ ]:
import csdmpy as cp
import matplotlib.pyplot as plt

import mrsimulator.signal_processing as sp
import mrsimulator.signal_processing.apodization as apo

from mrsimulator import Simulator, SpinSystem, Site
from mrsimulator.methods import BlochDecayCentralTransitionSpectrum

from lmfit import fit_report, Minimizer, Parameters

## Import the dataset
Use the [csdmpy](https://csdmpy.readthedocs.io/en/latest/index.html) module to load the synthetic dataset as a CSDM object.

In [ ]:
filename = "https://raw.githubusercontent.com/DeepanshS/mrsimulator-tutorial/master/resources/Rb_site.csdf"
synthetic_experiment = cp.load(filename)

# convert the dimension coordinates from Hz to ppm
synthetic_experiment.dimensions[0].to("ppm", "nmr_frequency_ratio")

# Normalize the spectrum
synthetic_experiment /= synthetic_experiment.max()

# Plot of the synthetic dataset.
ax = plt.subplot(projection="csdm")
ax.plot(synthetic_experiment.real, color="black", linewidth=1)
ax.set_xlim(20, -20)
ax.invert_xaxis()
plt.tight_layout()
plt.show()

## Create a fitting model

A fitting model is the combination of `Simulator` and `SignalProcessor` class objects.

#### Create the spin-system, method, and Simulator objects

In [ ]:
# the guess model comprising of a single site spin system
site = Site(
    isotope="87Rb",
    isotropic_chemical_shift=5,  # in ppm
    quadrupolar={"Cq": 1.5e6, "eta": 0.3},  # Cq is in Hz
)

system_object = SpinSystem(
    name="Rb Site",
    sites=[site],  # from the above code
)

In [ ]:
method = BlochDecayCentralTransitionSpectrum(
    channels=["87Rb"],
    magnetic_flux_density=9.4,  # in T
    rotor_frequency=14000,  # in Hz
    spectral_dimensions=[
        {
            "count": 2560,
            "spectral_width": 31237.7977352597,  # in Hz
            "reference_offset": 0,  # in Hz
        }
    ],
)

In [ ]:
sim = Simulator()
sim.spin_systems = [system_object]
sim.methods = [method]

sim.methods[0].experiment = synthetic_experiment

In [ ]:
sim.run()

Create a SignalProcessor class and apply post simulation processing.

In [ ]:
processor = sp.SignalProcessor(
    operations=[
        sp.IFFT(),
        apo.Gaussian(FWHM="20 Hz"),
        sp.FFT(),
        sp.Scale(factor=0.8),
    ]
)
processed_data = processor.apply_operations(data=sim.methods[0].simulation)

Plot of the spectrum

In [ ]:
ax = plt.subplot(projection="csdm")
ax.plot(processed_data.real, c="k", linewidth=1, label="guess spectrum")
ax.plot(synthetic_experiment.real, c="r", linewidth=1.5, alpha=0.5, label="experiment")
ax.set_xlim(20, -20)
ax.invert_xaxis()
plt.legend()
plt.tight_layout()
plt.show()

## Setup a Least-squares minimization

Now that our model is ready, the next step is to set up a least-squares minimization.
You may use any optimization package of choice, here we show an application using
LMFIT. You may read more on the LMFIT [documentation page](https://lmfit.github.io/lmfit-py/index.html).

### Create fitting parameters

Next, you will need a list of parameters that will be used in the fit. The *LMFIT*
library provides a [Parameters](https://lmfit.github.io/lmfit-py/parameters.html)
class to create a list of parameters.



In [ ]:
site1 = system_object.sites[0]
params = Parameters()

params.add(name="iso", value=site1.isotropic_chemical_shift)
params.add(name="eta", value=site1.quadrupolar.eta, min=0, max=1)
params.add(name="Cq", value=site1.quadrupolar.Cq)
params.add(name="FWHM", value=processor.operations[1].FWHM)
params.add(name="factor", value=processor.operations[3].factor)

### Create a minimization/loss function

Note, the above set of parameters does not know about the model. You will need to
set up a function that will

- update the parameters of the `Simulator` and `SignalProcessor` object based on the
  LMFIT parameter updates,
- re-simulate the spectrum based on the updated values, and
- return the difference between the experiment and simulation.



In [ ]:
def minimization_function(params, sim, processor):
    values = params.valuesdict()

    # the experiment data as a Numpy array
    intensity = sim.methods[0].experiment.dependent_variables[0].components[0].real

    # Here, we update simulation parameters iso, eta, and zeta for the site object
    site = sim.spin_systems[0].sites[0]
    site.isotropic_chemical_shift = values["iso"]
    site.quadrupolar.eta = values["eta"]
    site.quadrupolar.Cq = values["Cq"]

    # run the simulation
    sim.run()

    # update the SignalProcessor parameter and apply line broadening.
    # update the scaling factor parameter at index 3 of operations list.
    processor.operations[3].factor = values["factor"]
    # update the exponential apodization FWHM parameter at index 1 of operations list.
    processor.operations[1].FWHM = values["FWHM"]

    # apply signal processing
    processed_data = processor.apply_operations(sim.methods[0].simulation)

    # return the difference vector.
    return intensity - processed_data.dependent_variables[0].components[0].real

### Perform the least-squares minimization

With the synthetic dataset, simulation, and the initial guess parameters, we are ready
to perform the fit. To fit, we use the *LMFIT* [Minimizer](https://lmfit.github.io/lmfit-py/fitting.html) class.



In [ ]:
minner = Minimizer(minimization_function, params, fcn_args=(sim, processor))
result = minner.minimize()
print(fit_report(result))

#### The plot of the fit, measurement and the residuals is shown below.

In [ ]:
plt.figsize = (4, 3)
x, y_data = synthetic_experiment.to_list()
residual = result.residual
plt.plot(x, y_data.real, label="Spectrum")
plt.plot(x, y_data.real - residual, "r", alpha=0.5, label="Fit")
plt.plot(x, residual, alpha=0.5, label="Residual")

plt.xlabel("Frequency / Hz")
plt.xlim(20, -20)
plt.gca().invert_xaxis()
plt.grid(which="major", axis="both", linestyle="--")
plt.legend()
plt.tight_layout()
plt.show()

1. Hansen, M. R., Jakobsen, H. J., Skibsted, J., $^{29}\text{Si}$ Chemical Shift Anisotropies in Calcium Silicates from High-Field $^{29}\text{Si}$ MAS NMR Spectroscopy, Inorg. Chem. 2003, **42**, *7*, 2368-2377. [DOI: 10.1021/ic020647f](https://doi.org/10.1021/ic020647f)